In [1]:
%pip install -r requirements.txt 


Note: you may need to restart the kernel to use updated packages.


#### Setup Tracing

In [2]:
# Must precede any llm module imports
import os

from langtrace_python_sdk import langtrace

langtrace.init(api_key = os.environ.get("LANGTRACE_API_KEY"))

Initializing Langtrace SDK..
⭐ Leave our github a star to stay on top of our updates - https://github.com/Scale3-Labs/langtrace


/Users/amriksingh/dev/genai/crewai/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Exporting spans to Langtrace cloud..


#### Get the env variable from .env

In [3]:
# from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import LLM
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
# Warning control
import warnings
warnings.filterwarnings('ignore')

api_key = os.environ.get("GEMINI_API_KEY")

os.environ["GOOGLE_API_KEY"] = api_key
# llm=ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash",
#     verbose=True,
#     google_api_key=api_key
# )
model = "gemini-1.5-flash-002"
llm=LLM(model=f'gemini/{model}')



#### Initialize Gemini SDK

In [4]:
from crewai import Agent, Crew, Process, Task, LLM
import os
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
# print(api_key)
# llm = LLM(
#         model="gemini/gemini-1.5-flash",
#         api_key=api_key
#     )

# response = llm.generate_content("The opposite of hot is")
# print(response.text)

websiteSearchTool_config = dict(
        llm=dict(
            provider="google",
            config=dict(
                model=model,
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )

In [5]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

#### Load the agents and tasks

In [6]:

market_news_monitor_agent = Agent(
  role='Lead Market Analyst',
  goal="""
    Conduct in-depth, real-time analysis of financial news and market updates,
    providing insightful summaries to guide content creation.
  """,
  backstory="""
    As the Lead Market Analyst at a premier financial content platform, you
    specialize in dissecting market trends and economic shifts, ensuring
    that your content stays ahead of the curve and delivers the most
    relevant insights to your audience.
  """,
  allow_delegation= False,
  verbose= True,
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm,  # Optional
)


data_analyst_agent = Agent(
  role='Chief Data Strategist',
  goal="""
    Synthesize complex market data into actionable insights that can be
    transformed into compelling content.
  """,
  backstory="""
    As the Chief Data Strategist at a leading financial advisory firm,
    your expertise lies in analyzing vast datasets to uncover trends and
    opportunities that inform high-impact investment strategies.
  """,
  allow_delegation= False,
  verbose= True,
  tools=[SerperDevTool(), WebsiteSearchTool(config=websiteSearchTool_config)],
  llm=llm,  # Optional
)

content_creator_agent = Agent(
  role='Creative Content Director',
  goal="""
    Develop and oversee the creation of high-quality, engaging content that
    educates and captivates the target audience, with a focus on the latest
    financial trends and insights.
  """,
  backstory="""
    As the Creative Content Director at a top financial publishing house,
    you excel in crafting narratives that resonate with investors, blending
    in-depth analysis with compelling storytelling to create content that
    drives engagement and trust.
  """,
  allow_delegation= False,
  verbose= True,
  tools=[SerperDevTool(), WebsiteSearchTool(config=websiteSearchTool_config)],
  llm=llm,  # Optional
)

quality_assurance_agent = Agent(
  role='Chief Content Officer',
  goal="""
    Oversee and refine the content creation process, ensuring that all
    outputs are accurate, aligned with brand voice, and optimized for
    engagement.
  """,
  backstory="""
    You are the Chief Content Officer of a leading financial media company.
    Your role is to ensure that every piece of content not only meets high
    editorial standards but also delivers clear, actionable insights that
    can help your audience make informed decisions.
  """,
  allow_delegation= False,
  verbose= True,
  llm=llm,  # Optional
)

In [7]:

monitor_financial_news_task = Task(
    description="""
      Monitor and analyze the latest news and updates related to
      the financial markets, with a particular focus on {subject}.
      Identify and summarize the most relevant and impactful news items
      that could influence market trends or investor decisions.
      Utilize financial news APIs and real-time market data tools to
      gather up-to-date information. Focus on detecting trends,
      regulatory changes, or significant economic indicators that
      directly relate to {subject}.
    """,
    agent=market_news_monitor_agent,
    expected_output="""
      A detailed summary report highlighting the most impactful financial
      news and updates related to {subject}. This report should include
      key insights and their potential implications for the market and
      content strategy.
    """
)

analyze_market_data_task = Task(
    description="""
      Analyze market data and trends related to {subject}, with a focus
      on uncovering patterns, opportunities, and risks that could be
      leveraged in content creation. Use advanced data analytics tools
      to process large datasets and generate actionable insights.
      Pay special attention to how {subject} influences market movements,
      investor sentiment, and economic indicators.
    """,
    agent=data_analyst_agent,
    expected_output="""
      A comprehensive analysis report that highlights key market trends
      and actionable insights related to {subject}. The report should
      include data visualizations and clear recommendations for content
      creation.
    """
)

create_content_task = Task(
    description="""
      Based on the insights provided by the Market News Monitor and
      Data Analyst agents, create high-quality, engaging content that
      educates and informs the target audience about {subject}.
      Produce various types of content, including blog posts and
      social media updates, that effectively communicate the insights
      gathered. Ensure the content clearly conveys the key findings
      and recommendations related to {subject}. Incorporate data
      visualizations, infographics, or other multimedia elements to
      enhance the content where applicable.
    """,
    agent=content_creator_agent,
    expected_output="""
      A collection of high-quality content pieces related to {subject},
      including blog posts and social media updates, complete with
      integrated multimedia elements.
    """,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    description="""
      Review and refine the content created on {subject} to ensure it
      meets the highest standards of accuracy, clarity, and brand
      alignment. Thoroughly proofread and edit the content, checking
      for errors, inconsistencies, and alignment with the brand voice.
      Ensure that the content accurately reflects the key insights and
      recommendations provided by the Data Analyst and Market News
      Monitor agents. Ensure that the final content is well-formatted
      in markdown, using appropriate headers, bullet points, links,
      and other markdown features to enhance readability and engagement.
    """,
    agent=quality_assurance_agent,
    expected_output="""
      A finalized set of content pieces related to {subject}, thoroughly
      reviewed, and formatted in markdown. The content should be
      well-structured, with appropriate use of headers, bullet points,
      links, and other markdown features to ensure it is both visually
      appealing and easy to read. Each section should have at least 2
      paragraphs.
    """,
    output_pydantic=ContentOutput
)

In [8]:
# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

In [9]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'world cup and qatar real estate'
})

# Agent: Lead Market Analyst
## Task: 
      Monitor and analyze the latest news and updates related to
      the financial markets, with a particular focus on world cup and qatar real estate.
      Identify and summarize the most relevant and impactful news items
      that could influence market trends or investor decisions.
      Utilize financial news APIs and real-time market data tools to
      gather up-to-date information. Focus on detecting trends,
      regulatory changes, or significant economic indicators that
      directly relate to world cup and qatar real estate.
    


# Agent: Lead Market Analyst
## Thought: Thought: I need to search for recent news and updates regarding the impact of the World Cup on Qatar's real estate market.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"impact of world cup on qatar real estate market\"}"
## Tool Output: 

Search results: Title: [PDF] Impact of the FIFA World Cup on Qatar: A look at the real estate ...
Li

In [12]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']

for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)